In [7]:
import matplotlib.pyplot as plt
import cv2
import random
import time
import numpy as np
import math

In [8]:
def get_rand_vector():
     return [random.uniform(-1, 1), random.uniform(-1, 1)]

In [9]:
class Rocket:
    def __init__(self, dna=None):
        self.pos = [canvas_width // 2, canvas_height // 2]
        self.vel = get_rand_vector()
        self.acc = [0, 0]
        if dna is None:
            self.dna = DNA()
        else:
            self.dna = dna
        self.count = 0
        self.fitness = 0
        self.crashed = False
    
    def apply_force(self, force):
        new_acc = [force_val + acc_val for force_val, acc_val in zip(force, self.acc)]
        self.acc = new_acc
    
    def update(self):
        self.apply_force(self.dna.genes[self.count])
        
        if self.crashed:
            self.vel = [0 for i in range(len(self.vel))]
        else:
            new_vel = [acc_val + vel_val for acc_val, vel_val in zip(self.acc, self.vel)]
            self.vel = new_vel
        
        new_pos = [vel_val + pos_val for vel_val, pos_val in zip(self.vel, self.pos)]
        new_x, new_y = new_pos
        
        if new_x < 0 or new_x >= canvas_width or new_y < 0 or new_y >= canvas_height or (new_x >= obs_x and new_x <= obs_x + obs_w and new_y >= obs_y and new_y <= obs_y + obs_h):
            new_x = max(min(new_x, canvas_width - 1), 0)
            new_y = max(min(new_y, canvas_height - 1), 0)
            new_pos = [new_x, new_y]
            self.crashed = True
            
        self.pos = new_pos
        self.acc = [0 for i in range(len(self.acc))]
        self.calc_fitness()
        
        self.count += 1
        
    def draw(self):
        pos_x = int(self.pos[0])
        pos_y = int(self.pos[1])
        canvas[pos_y, pos_x] = 1
            
    def calc_fitness(self):
        cur_x, cur_y = self.pos
        dist = math.sqrt((cur_x - target_x)**2 + (cur_y - target_y)**2)
        
        if dist < dist_thresh:
            if dist == 0:
                self.fitness = 1
            params['complete'] = True
        else:
            self.fitness = 1 / dist
        
        if self.crashed:
            self.fitness = 0

In [19]:
class Population():
    def __init__(self):
        self.rockets = []
        self.mating_pool = []
    
        for i in range(pop_size):
            self.rockets.append(Rocket())
            
    def run(self):
        for rocket in self.rockets:
            rocket.update()
            rocket.draw()
            
    def evaluate(self):
        max_fit = 0
        for rocket in self.rockets:
            rocket.calc_fitness()
            if rocket.fitness > max_fit:
                max_fit = rocket.fitness
        if max_fit == 0:
            max_fit = 1
            
        for rocket in self.rockets:
            scaled_fitness = int(math.pow(rocket.fitness / max_fit, 4) * 100)
            for j in range(scaled_fitness):
                self.mating_pool.append(rocket)
                
    def selection(self):
        new_rockets = []
        if len(self.mating_pool) == 0:
            self.rockets = []
            for i in range(pop_size):
                self.rockets.append(Rocket())
        else:
            for i in range(pop_size):
                dad = self.mating_pool[int(random.uniform(0, len(self.mating_pool)))].dna
                mom = self.mating_pool[int(random.uniform(0, len(self.mating_pool)))].dna
                child = dad.crossover(mom)
                child.mutation()
                new_rocket = Rocket(child)
                new_rockets.append(new_rocket)
            self.rockets = new_rockets

In [20]:
class DNA:
    def __init__(self):
        self.genes = [get_rand_vector() for i in range(lifespan)]
        
    def crossover(self, dna2):
        new_dna = DNA()
        mid = len(new_dna.genes) // 2
        new_dna.genes[0:mid] = self.genes[0:mid]
        new_dna.genes[mid:] = dna2.genes[mid:]
        return new_dna
    
    def mutation(self):
        for i in range(len(self.genes)):
            if random.uniform(0, 1) < mut_value:
                self.genes[i] = get_rand_vector()

In [22]:
mut_value = 0.1
lifespan = 400
pop_size = 1000
dist_thresh = 5
params = { 'complete': False }
generations = 0
i = 0
canvas_width = 250
canvas_height = 250
obs_x, obs_y, obs_w, obs_h = [25, 200, 150, 10]
target_x, target_y = [canvas_width // 2, canvas_height - 10]
population = Population()
while True:
    canvas = np.zeros((canvas_height, canvas_width))
    cv2.rectangle(canvas, (target_x - 1, target_y - 1), (target_x + 1, target_y + 1), (255, 0, 0), -1)
    cv2.rectangle(canvas, (obs_x, obs_y), ((obs_x + obs_w), (obs_y + obs_h)), (255, 0, 0), -1)
    cv2.putText(canvas, 'Generation: {}'.format(generations), (20, 20), cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
#     canvas[target_y, target_x] = 1
    population.run()
    cv2.imshow('canvas', canvas)
    i += 1
    if i == lifespan or params['complete']:
        population.evaluate()
        population.selection()
        params['complete'] = False
        i = 0
        generations += 1
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()